In [1]:
import numpy as np
import os
import time
import glob
import copy
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from getImg import *
from makeLgMatrix import *
from bkgSubtraction import *
from rebinImg import *
from centering import *
from getImgNorm import *
from getImgInfo import *
from fitLegendres import *
from saveResults import *
from queryFolder import *
from onlineAnalysis.py import onlineAnalysis

In [2]:
class CONFIG():
  def __init__(self):


    ###################################
    ###  file querying information  ###
    ###################################

    self.doQueryFolder  = True
    self.queryFolder    = "/reg/ued/ana/scratch/CHD/20161212/testScan"
    self.queryBkgAddr   = "/reg/ued/ana/scratch/CHD/20161212/LongScan2/run013/images-ANDOR1/ANDOR1_delayHigh-001-024.4950_0001.tif"


    ########################
    ###  saving results  ###
    ########################

    self.saveQueryResults   = False
    self.saveLoadedResults  = False
    self.saveFolder         = "results"
    self.saveFileName       = "fullResults"


    ##################################
    ###  loading results and data  ###
    ##################################

    # load previous data
    self.loadFolders    = []       # do not load anything if empty
    self.fileExtention  = ".tif"
    """
    badRuns = ["096","111","1i18","010","138","149","035","039","077","007","086","008","091","026"]
    for fld in glob.glob("/reg/ued/ana/scratch/CHD/20161212/LongScan2/run*"):
      skip = False
      for i in badRuns:
        if "run"+i in fld:
          skip = True
          break
      if skip:
        continue

      ind = fld.find("run")
      runNum = int(fld[ind+3:])
      if True: #(runNum > 80) and (runNum <= 180):

        print(fld)
        self.loadFolders.append({
            "folder": fld + "/images-ANDOR1", 
            "background": None,
            #"background": "/reg/ued/ana/scratch/CHD/20161212/LongScan2/run013/images-ANDOR1/ANDOR1_delayHigh-001-024.4950_0001.tif",
            "centerR" : 550,
            "centerC" : 508})
    """

    # load saved results (saved by this analysis)
    self.loadSavedResults           = False
    self.loadSavedResultsFolder     = "results"
    self.loadSavedResultsFileName   = "allCHD_"


    #############################
    ###  analysis parameters  ###
    #############################

    # background images (will sum images in list)
    self.bkgImgNames = [ 
          "/reg/ued/ana/scratch/CHD/20161212/LongScan2/run013/images-ANDOR1/ANDOR1_delayHigh-001-024.4950_0001.tif",
          "/reg/ued/ana/scratch/CHD/20161212/LongScan2/run013/images-ANDOR1/ANDOR1_delayHigh-002-024.5850_0001.tif"]

    # hot pixels
    self.hotPixel = 7000

    # image region of interest
    self.roi = 834

    # readout noise subtraction range
    self.ROradLow   = 0.9
    self.ROradHigh  = 0.98
    
    # image normalization range
    self.normRadLow   = 0.65     # [0.5, 0.75]
    self.normRadHigh  = 0.95     # [0.9, 0.99]
    self.sumMin       = 5.59e8   # ignore all images with a total sum below this
    self.sumMax       = 5.72e8   # ignore all images with a total sum above this

    # center finding
    self.centerR        = 550    # use None to do center finding  
    self.centerC        = 508    # use None to do center finding  [480, 520]
    self.guessCenterR   = 530    # [530, 570]
    self.guessCenterC   = 500    # [480, 520]
    self.centerRadLow   = 148    # use this range to determine center
    self.centerRadHigh  = 152    # use this range to determine center

    # legendre fitting parameters
    self.Nlegendres     = 6      # will fit legendres [0, Nlegendres)
    self.NradialBins    = 50     # number of radial bins when fitting
    self.Nrebin         = 5      # rebin before fitting (makes fitting faster)
    self.gMatrixFolder  = "/reg/neh/home/khegazy/analysis/legendreFitMatrices/"

    
    

In [3]:
testImages = []
backgroundImage = 

centerConfig = copy.deepcopy(config)
centerConfig.centerR = None
centerConfig.centerC = None

bkgImg = get_image(backgroundImage, config.hotPixel)

for name in testImages:
    ###  get image and remove hot pixels  ###
    img = get_image(name, config.hotPixel)

    ###  subtract background images  ###
    if curBkgAddr is not None:
        img -= bkgImg #background_subtraction(img, bkgImg)
    
    ###  center image  ###
    img, centerR, centerC = centering(img, centerConfig)
    
    print("Found image center at row/column %i / %i" % (centerR, centerC))
    
    fig,ax = plt.subplot(111)
    ax.imshow(img)

    

SyntaxError: invalid syntax (<ipython-input-3-f119c3200080>, line 2)

In [4]:
imgSums = onlineAnalysis(config, getImgNormDistribution=True)

fig = plt.figure()
plt.hist(imgSums)
plt.show()

NameError: name 'onlineAnalysis' is not defined

In [ ]:
onlinAnalysis(config)